In [ ]:
%pip install gurobipy
!pip install ortools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 27.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.24.2 which is incompatible.


In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [ ]:
city_names = ["City 0", "City 1", "City 2", "City 3", "City 4"]
city_coordinates = [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]
num_cities = len(city_names)

In [ ]:
def euclidean_distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    return int((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

distance_matrix = {}
for i in range(num_cities):
    distance_matrix[i] = {}
    for j in range(num_cities):
        if i != j:
            distance_matrix[i][j] = euclidean_distance(city_coordinates[i], city_coordinates[j])

In [ ]:
def create_data_model():
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0  # Starting city
    return data

data = create_data_model()

In [ ]:
def solve_tsp(data):
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    solution = routing.SolveWithParameters(search_parameters)
    return solution, manager, routing

solution, manager, routing = solve_tsp(data)

In [ ]:
def print_solution(manager, routing, solution):
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(city_names[manager.IndexToNode(index)])
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(city_names[manager.IndexToNode(index)])
    route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    print(plan_output)
    print('Route distance: {} miles'.format(route_distance))

if solution:
    print_solution(manager, routing, solution)


Objective: 0 miles
Route:
 City 0 -> City 4 -> City 3 -> City 2 -> City 1 -> City 0

Route distance: 0 miles


In [ ]:
import itertools

# List of cities
cities = ['A', 'B', 'C', 'D', 'E']

# Initialize the constraint counter
constraint_count = 0

# Iterate through all combinations of cities (excluding the initial city)
for i, j in itertools.combinations(cities[1:], 2):
    # Add a subtour elimination constraint
    constraint_count += 1
    print(f"Constraint {constraint_count}: x_{i}{j} + x_{j}{i} <= {len(cities) - 2}")

print(f"Total subtour elimination constraints: {constraint_count}")

Constraint 1: x_BC + x_CB <= 3
Constraint 2: x_BD + x_DB <= 3
Constraint 3: x_BE + x_EB <= 3
Constraint 4: x_CD + x_DC <= 3
Constraint 5: x_CE + x_EC <= 3
Constraint 6: x_DE + x_ED <= 3
Total subtour elimination constraints: 6


In [ ]:
from ortools.linear_solver import pywraplp

# Define the distance matrix
distances = [
    [0, 10, 8, 9, 7],
    [10, 0, 10, 5, 6],
    [8, 10, 0, 8, 9],
    [9, 5, 8, 0, 6],
    [7, 6, 9, 6, 0]
]

num_cities = len(distances)
num_routes = num_cities * (num_cities - 1)

# Create a solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Create binary decision variables for each edge (route)
x = {}
for i in range(num_cities):
    for j in range(num_cities):
        if i != j:
            x[i, j] = solver.IntVar(0, 1, f'x_{i}_{j}')

# Create objective function: minimize total distance
objective = solver.Objective()
for i in range(num_cities):
    for j in range(num_cities):
        if i != j:
            objective.SetCoefficient(x[i, j], distances[i][j])
objective.SetMinimization()

# Add constraints: ensure each city is visited exactly once
for i in range(num_cities):
    constraint = solver.Constraint(1, 1)
    for j in range(num_cities):
        if i != j:
            constraint.SetCoefficient(x[i, j], 1)

# Solve the TSP
solver.Solve()

# Extract the optimal route
route = []
start_city = 0
while True:
    for j in range(num_cities):
        if start_city != j and x[start_city, j].solution_value() == 1:
            route.append(start_city)
            start_city = j
            break
    if len(route) == num_cities:
        break

# Add the last city to complete the loop
route.append(start_city)

# Calculate the total distance of the optimal route
total_distance = sum(distances[route[i]][route[i + 1]] for i in range(num_cities - 1))
total_distance += distances[route[-1]][route[0]]  # Add distance to return to the starting city

print("Optimal Route:", "->".join([str(city) for city in route]))
print("Total Distance:", total_distance)


Optimal Route: 0->4->1->3->1->3
Total Distance: 32
